In [1]:
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize
import numpy as np
import matplotlib.pyplot as plt
import time



def Simulator_v3(td, tm, activity, test=False, activities=[100, 100], tss=[0, 0], final_values=False):
    '''
    This function, Simulator, calculates the number of the atoms at the measurement time, tm, for 
    the case that the person has died at time td. The unit of the time values is day.
    This function solves a set if differential equations to find the number if the atoms.
    '''
    
    decay_time = td
    time_of_measurement = tm
    
    
    lds = [0.18, 0.000085, 0.139, 0.005007]
    Activity = activity

    #--This is for calculation of # of atoms for lead over time e.g. a month
    if(test):
        N0_Rn = 0
    else:
        N0_Rn = (int(Activity) * 3.8 * 24 * 3600)/0.693
    N0_Pb = 0
    N0_Bi = (lds[1]/lds[2])*N0_Pb
    N0_Po = (lds[1]/lds[3])*N0_Pb
    
    tss = np.append(tss, td)
    def acts(t):
        for i in range(1, len(tss)):
            if(t<tss[i])and(t>=tss[i-1]):
                return activities[i-1]
        return activities[-1]
    
    
    # Funtion f(t, N) and is the function that describes the differential equation:
    # f(t, N) = dN/dt where N is an array the number of atoms for different compounds.
    def f(t, N):
        if(test):
            act = acts(t) if t<=decay_time else 0
            #N1decay = 0 if t<decay_time else lds[0]
            return [act-lds[0]*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]
        else:
            N1decay = 0 if t<decay_time else lds[0]
            return [-N1decay*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]

    
    ts = 100
    n = 500
    # C1 and C2 are the number of atoms that we measure in reality.
    time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
    
    # solve_ivp solves the set of differential equations.
    #output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12)
    output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12, t_eval=time)
    
    '''
    if final_values:
        C1 = np.interp(tm, output.t, output.y[1])
        C2 = np.interp(tm, output.t, output.y[2])
        C3 = np.interp(tm, output.t, output.y[3])
        R1 = C1/C3
        R2 = C1/C2
        return tm, R1, R2
    else:
        #ts = 2000
        #n = 500
        # C1 and C2 are the number of atoms that we measure in reality.
        #time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
        #time = np.linspace(ts, tm, n)
        C1 = np.interp(time, output.t, output.y[1])
        C2 = np.interp(time, output.t, output.y[2])
        C3 = np.interp(time, output.t, output.y[3])
        #C1 = output.y[1]
        #C2 = output.y[2]
        #C3 = output.y[3]
        R1 = C1/C3
        R2 = C1/C2
        return time, R1, R2
        #return output.t, R1, R2
    '''
    C1 = output.y[1]
    C2 = output.y[2]
    C3 = output.y[3]
    return output.t, C1/C3, C1/C2


def inverseProblemSolver3(R1t, R2t):   
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.02, 0.01]
    def objective(inp, depth=0):
        td, ms, jump, tc = inp
        if(ms<=0)or(td<=1)or(tc<0)or(tc>tcs[depth])or(jump<=0.1)or(jump>1.8):
            return np.inf
        Rd_act = 100
        _, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        #_, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        #print(inp)
        return (((R1t-r1[-1])/R1t)**2 + 10*((R2t-r2[-1])/R2t)**2)**0.5
    
    for depth in range(len(tcs)):
        print(depth)
        #solution = minimize(objective, x0=[15000, 10, 0.8, 0.99*tcs[depth]], method='Nelder-Mead', tol=1e-9, args=depth)
        solution = minimize(objective, x0=[15000, 10, 0.4, 0.99*tcs[depth]], method='Nelder-Mead', args=depth,
                            options={'fatol': 1e-9, 'xatol': 1e-4})
        if(solution.success): 
            break
    print('depth: {}'.format(depth))
    return solution


def inverseProblemSolver3_DoubleMeasurement(R1t1, R2t1, R1t2, R2t2, step):
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.02, 0.01]
    def objective(inp, depth=0):
        td, tm, jump, tc = inp
        if(tm<=0)or(td<=1)or(tc<0)or(tc>tcs[depth])or(jump<=0.1)or(jump>1.8):
            return np.inf
        Rd_act = 100
        t, r12, r22 = Simulator_v3(td, td+tm+step, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        r11 = np.interp(td+tm, t, r12)
        r21 = np.interp(td+tm, t, r22)
        #_, r1, r2 = Simulator_v3(td, td+tm, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        #print(inp)
        return (((R1t1-r11)/R1t1)**2 + ((R1t2-r12[-1])/R1t2)**2 + 20*(((R2t1-r21)/R2t1)**2 + ((R2t2-r22[-1])/R2t2)**2))**0.5
    
    best_sol = None
    for depth in range(len(tcs)):
        print(depth)
        #solution = minimize(objective, x0=[15000, 10, 0.8, 0.99*tcs[depth]], method='Nelder-Mead', tol=1e-9, args=depth)
        solution = minimize(objective, x0=[15000, 10, 0.4, 0.99*tcs[depth]], method='Nelder-Mead', args=depth,
                            options={'fatol': 1e-9, 'xatol': 1e-6})
        if(solution.success)and(abs(solution.fun)<1e-7):
            print(solution.fun)
            best_sol = solution
            break
        else:
            if(best_sol is None):
                best_sol = solution
            elif(abs(solution.fun)<abs(best_sol.fun)):
                best_sol = solution
            print(solution.fun)
    print('depth: {}'.format(depth))
    return best_sol


def inverseProblemSolver4(R1t1, R1t2, R2t1, R2t2, step):
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.5, 0.1, 0.01, 0.001]
    tm_guess = np.arange(3, 22, 3)
    def objective(inp, i=0, j=0):
        td, tm, jump, tc = inp
        out_of_bound = (tm<=(0 if i==0 else tm_guess[i-1]))or(tm>tm_guess[i])or(td<=1)or(tc<=0)or(tc>tcs[j])or(jump<=0.1)or(jump>1.8)
        if out_of_bound:
            return np.inf
        Rd_act = 100
        t, r1, r2 = Simulator_v3(td, td+tm+step, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        #_, r1, r2 = Simulator_v3(td, td+tm, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        r11, r12 = np.interp([td+tm, td+tm+step], t, r1)
        r21, r22 = np.interp([td+tm, td+tm+step], t, r2)
        return (((R1t1-r11)/R1t1)**2 + ((R1t2-r12)/R1t2)**2 + 10*(((R2t1-r21)/R2t1)**2 + ((R2t2-r22)/R2t2)**2))**0.5
    
    min_sol = None
    min_fun = np.inf
    solved = False
    for j in range(len(tcs)):
        for i in range(len(tm_guess)):
            print("initial guesses: {}\t{}".format(tm_guess[i], tcs[j]))
            solution = minimize(objective, x0=[15000, tm_guess[i], 0.4, 0.999*tcs[j]], method='Nelder-Mead', args=(i, j),
                                options={'fatol': 1e-9, 'xatol': 1e-4})
            if(solution.success):
                if(abs(solution.fun)<min_fun):
                    min_sol = solution
                if(abs(solution.fun)>1e-9):
                    print("Function magnitude: " + str(abs(solution.fun)) + "\ttm: " + str(solution.x[1]))
                    print("Resetting the initial guess...")
                else:
                    solved = True
                    break
        if(solved):
            break
    if not solved:
        solutoin = min_sol
    return solution

In [14]:
############ Compairing previous single measurement method with double measurement method ################

from datetime import datetime

files = ['Data_report_no_error_v2_2023_12_18_19_23_53.npy']

data = []

for file in files:
    subdata = np.load(file, allow_pickle=True)
    for j in range(len(subdata)):
        data.append(subdata[j])

        
#data = data[:1]
        
step = 3
total_time = 0
new_data = []
errs = np.zeros(len(data))
tms = np.zeros(len(data))
results = np.zeros(len(data), dtype='bool')

for i in range(len(data)):
    start = time.time()
    
    t, R1t, R2t = Simulator_v3(data[i]['td'], data[i]['td']+data[i]['tm']+step,
                               100, True, data[i]['Rd'], data[i]['Tjumps'])

    Rt11 = np.interp(data[i]['td']+data[i]['tm'], t, R1t)
    Rt21 = np.interp(data[i]['td']+data[i]['tm'], t, R2t)
    Rt12 = R1t[-1]
    Rt22 = R2t[-1]

    sol = inverseProblemSolver3_DoubleMeasurement(Rt11, Rt21, Rt12, Rt22, step)
    
    errs[i] = np.abs(data[i]['tm']-sol.x[1])
    tms[i] = data[i]['tm']
    results[i] = sol.success
    
    s = {'td':data[i]['td'], 'tm':data[i]['tm'], 'Rd':data[i]['Rd'], 'Tjumps':data[i]['Tjumps'], 'sol':sol}
    
    new_data.append(s)
    
    stop = time.time()
    print('{}/{}\t{:.2f}\t{}\t{:.2f} seconds'.format(i + 1, len(data), errs[i], results[i], stop-start))
    total_time += stop-start
        
    remaining = total_time * (len(data) - (i+1))/(i+1)
    remtime = [int(remaining//3600), int((remaining%3600)//60), int(remaining%60)]
    print('Estimated remaining time: {} hours {} minutes {} seconds\n\n'.format(remtime[0], remtime[1], remtime[2]))

now = datetime.now()    
np.save('Data_report_no_error_v2_double_measurement_{}_{}_{}_{}_{}_{}.npy'.format(now.year, now.month, now.day, now.hour, now.minute, now.second), new_data)

%matplotlib qt
plt.plot(tms, np.abs(errs), '.')
plt.xlabel('Time difference between death and measurement [days]')
plt.ylabel('Estimation error [days]')
plt.grid()
plt.show()

plt.show()

0
1.5708522474125591e-10
depth: 0
1/1000	0.00	True	35.08 seconds
Estimated remaining time: 9 hours 44 minutes 9 seconds


0
1.7486625939773765e-11
depth: 0
2/1000	0.00	True	51.25 seconds
Estimated remaining time: 11 hours 57 minutes 58 seconds


0
4.2358780222041064e-11
depth: 0
3/1000	0.00	True	39.36 seconds
Estimated remaining time: 11 hours 36 minutes 9 seconds


0
2.7731777323779348e-08
depth: 0
4/1000	0.00	True	38.25 seconds
Estimated remaining time: 11 hours 20 minutes 20 seconds


0
1.2799143529116612e-10
depth: 0
5/1000	0.00	True	38.86 seconds
Estimated remaining time: 11 hours 12 minutes 37 seconds


0
1.597801342327717e-10
depth: 0
6/1000	0.00	True	41.01 seconds
Estimated remaining time: 11 hours 13 minutes 10 seconds


0
6.185416799811205e-11
depth: 0
7/1000	0.00	True	32.40 seconds
Estimated remaining time: 10 hours 53 minutes 1 seconds


0
2.649993567462308e-10
depth: 0
8/1000	0.00	True	41.69 seconds
Estimated remaining time: 10 hours 56 minutes 59 seconds


0
5.03744556625

3.6510609049456844e-08
depth: 0
66/1000	0.02	True	31.14 seconds
Estimated remaining time: 11 hours 14 minutes 13 seconds


0
3.529144476281773e-11
depth: 0
67/1000	0.00	True	39.27 seconds
Estimated remaining time: 11 hours 12 minutes 33 seconds


0
4.549923768593737e-11
depth: 0
68/1000	0.00	True	34.14 seconds
Estimated remaining time: 11 hours 9 minutes 45 seconds


0
3.5765437775250934e-07
1
3.8220364210062455e-07
2
0.0020365030156121334
3
3.5738292858696135e-07
4
3.5737379505652715e-07
5
3.573760113052057e-07
6
3.5741454482443156e-07
depth: 6
69/1000	0.00	True	210.93 seconds
Estimated remaining time: 11 hours 46 minutes 46 seconds


0
2.4934254311883414e-11
depth: 0
70/1000	0.00	True	40.95 seconds
Estimated remaining time: 11 hours 44 minutes 59 seconds


0
2.6746597884139525e-11
depth: 0
71/1000	0.00	True	64.85 seconds
Estimated remaining time: 11 hours 48 minutes 27 seconds


0
2.9230239516695786e-11
depth: 0
72/1000	0.00	True	70.59 seconds
Estimated remaining time: 11 hours 53 mi

1.2793124333752522e-10
depth: 0
129/1000	0.00	True	35.33 seconds
Estimated remaining time: 11 hours 50 minutes 15 seconds


0
1.564864258777898e-10
depth: 0
130/1000	0.00	True	52.91 seconds
Estimated remaining time: 11 hours 49 minutes 53 seconds


0
5.497935263039304e-11
depth: 0
131/1000	0.00	True	27.64 seconds
Estimated remaining time: 11 hours 46 minutes 43 seconds


0
1.1056152382259467e-10
depth: 0
132/1000	0.00	True	33.13 seconds
Estimated remaining time: 11 hours 44 minutes 11 seconds


0
4.440256885238048e-10
depth: 0
133/1000	0.00	True	42.71 seconds
Estimated remaining time: 11 hours 42 minutes 43 seconds


0
3.122510030634119e-10
depth: 0
134/1000	0.00	True	30.44 seconds
Estimated remaining time: 11 hours 39 minutes 57 seconds


0
5.1186995945185466e-11
depth: 0
135/1000	0.00	True	34.65 seconds
Estimated remaining time: 11 hours 37 minutes 40 seconds


0
6.518833215798618e-08
depth: 0
136/1000	0.05	True	30.71 seconds
Estimated remaining time: 11 hours 34 minutes 59 seconds



1.1261404215366276e-09
depth: 0
192/1000	0.00	True	40.81 seconds
Estimated remaining time: 10 hours 57 minutes 0 seconds


0
2.773646890804845e-10
depth: 0
193/1000	0.00	True	37.69 seconds
Estimated remaining time: 10 hours 55 minutes 25 seconds


0
1.0275155918827167e-10
depth: 0
194/1000	0.00	True	33.23 seconds
Estimated remaining time: 10 hours 53 minutes 32 seconds


0
2.4725782965618334e-10
depth: 0
195/1000	0.00	True	41.31 seconds
Estimated remaining time: 10 hours 52 minutes 13 seconds


0
1.0339316211941339e-08
depth: 0
196/1000	0.00	True	52.08 seconds
Estimated remaining time: 10 hours 51 minutes 38 seconds


0
5.0716815813312164e-11
depth: 0
197/1000	0.00	True	31.99 seconds
Estimated remaining time: 10 hours 49 minutes 42 seconds


0
5.241404596925643e-11
depth: 0
198/1000	0.00	True	31.01 seconds
Estimated remaining time: 10 hours 47 minutes 42 seconds


0
7.482393427365553e-11
depth: 0
199/1000	0.00	True	31.93 seconds
Estimated remaining time: 10 hours 45 minutes 47 seconds


2.0366044311467663e-10
depth: 0
255/1000	0.00	True	38.29 seconds
Estimated remaining time: 10 hours 17 minutes 17 seconds


0
4.580523362444808e-11
depth: 0
256/1000	0.00	True	37.49 seconds
Estimated remaining time: 10 hours 15 minutes 52 seconds


0
6.52688839491521e-08
depth: 0
257/1000	0.01	True	93.02 seconds
Estimated remaining time: 10 hours 17 minutes 7 seconds


0
3.805951005248872e-11
depth: 0
258/1000	0.00	True	31.94 seconds
Estimated remaining time: 10 hours 15 minutes 26 seconds


0
1.423202953499265e-09
depth: 0
259/1000	0.00	True	64.37 seconds
Estimated remaining time: 10 hours 15 minutes 18 seconds


0
4.178952442239793e-08
depth: 0
260/1000	0.02	True	26.28 seconds
Estimated remaining time: 10 hours 13 minutes 21 seconds


0
3.113411511891497e-11
depth: 0
261/1000	0.00	True	38.60 seconds
Estimated remaining time: 10 hours 12 minutes 0 seconds


0
2.735010628525792e-09
depth: 0
262/1000	0.00	True	35.22 seconds
Estimated remaining time: 10 hours 10 minutes 30 seconds


0
8.

...

5.234494666595439e-08
depth: 0
702/1000	0.00	True	27.26 seconds
Estimated remaining time: 4 hours 8 minutes 5 seconds


0
1.90700469723824e-09
depth: 0
703/1000	0.00	True	33.63 seconds
Estimated remaining time: 4 hours 7 minutes 8 seconds


0
9.25602688328534e-09
depth: 0
704/1000	0.00	True	40.78 seconds
Estimated remaining time: 4 hours 6 minutes 14 seconds


0
3.039967511913049e-07
1
8.387090599989505e-08
depth: 1
705/1000	0.00	True	44.72 seconds
Estimated remaining time: 4 hours 5 minutes 22 seconds


0
1.1939706616777107e-07
1
1.0735598395385482e-07
2
1.194843174032293e-07
3
1.1948799246672446e-07
4
1.1947763548698302e-07
5
1.1947582362472362e-07
6
1.194912870807466e-07
depth: 6
706/1000	0.00	True	231.50 seconds
Estimated remaining time: 4 hours 5 minutes 48 seconds


0
1.4625350109465757e-10
depth: 0
707/1000	0.00	True	34.43 seconds
Estimated remaining time: 4 hours 4 minutes 51 seconds


0
5.58889668055817e-11
depth: 0
708/1000	0.00	True	45.23 seconds
Estimated remaining time: 4 

9.723996683333424e-11
depth: 0
764/1000	0.00	True	35.18 seconds
Estimated remaining time: 3 hours 19 minutes 7 seconds


0
3.289751661940185e-11
depth: 0
765/1000	0.00	True	29.82 seconds
Estimated remaining time: 3 hours 18 minutes 10 seconds


0
6.826955289555486e-11
depth: 0
766/1000	0.00	True	37.26 seconds
Estimated remaining time: 3 hours 17 minutes 15 seconds


0
3.636838648262315e-09
depth: 0
767/1000	0.00	True	43.63 seconds
Estimated remaining time: 3 hours 16 minutes 22 seconds


0
2.392914154940783e-12
depth: 0
768/1000	0.00	True	68.34 seconds
Estimated remaining time: 3 hours 15 minutes 37 seconds


0
4.183996715399534e-08
depth: 0
769/1000	0.00	True	26.94 seconds
Estimated remaining time: 3 hours 14 minutes 39 seconds


0
1.5139054336317058e-07
1
1.303171859424934e-07
2
1.2892814178750356e-07
3
1.2879228209565508e-07
4
1.2878428023299294e-07
5
1.2878294099693606e-07
6
1.2878582706947751e-07
depth: 6
770/1000	0.01	True	171.76 seconds
Estimated remaining time: 3 hours 14 minut

4.0021771329672965e-07
1
4.002208192889396e-07
2
4.002129814834108e-07
3
4.00218373784509e-07
4
4.0021756507456153e-07
5
4.002185320255523e-07
6
4.002183200058513e-07
depth: 6
825/1000	0.01	True	310.07 seconds
Estimated remaining time: 2 hours 29 minutes 14 seconds


0
6.167390220298705e-09
depth: 0
826/1000	0.00	True	52.37 seconds
Estimated remaining time: 2 hours 28 minutes 23 seconds


0
9.194032859595425e-09
depth: 0
827/1000	0.01	True	42.52 seconds
Estimated remaining time: 2 hours 27 minutes 30 seconds


0
4.794487157504123e-11
depth: 0
828/1000	0.00	True	40.49 seconds
Estimated remaining time: 2 hours 26 minutes 37 seconds


0
1.6550614676466886e-11
depth: 0
829/1000	0.00	True	36.86 seconds
Estimated remaining time: 2 hours 25 minutes 43 seconds


0
2.8882521715969006e-11
depth: 0
830/1000	0.00	True	42.75 seconds
Estimated remaining time: 2 hours 24 minutes 50 seconds


0
2.378765336672873e-06
1
0.004537624993031091
2
2.736354695286743e-06
3
2.5682537397087028e-06
4
2.3507650371

6.853332080768485e-09
depth: 0
888/1000	0.00	True	29.16 seconds
Estimated remaining time: 1 hours 34 minutes 47 seconds


0
4.715532253447127e-09
depth: 0
889/1000	0.00	True	25.60 seconds
Estimated remaining time: 1 hours 33 minutes 53 seconds


0
4.31441497062811e-11
depth: 0
890/1000	0.00	True	33.25 seconds
Estimated remaining time: 1 hours 33 minutes 0 seconds


0
6.635511812640829e-10
depth: 0
891/1000	0.00	True	50.12 seconds
Estimated remaining time: 1 hours 32 minutes 9 seconds


0
3.204481885033382e-11
depth: 0
892/1000	0.00	True	38.80 seconds
Estimated remaining time: 1 hours 31 minutes 17 seconds


0
4.1559782556949335e-08
depth: 0
893/1000	0.01	True	23.15 seconds
Estimated remaining time: 1 hours 30 minutes 23 seconds


0
2.2420427818021674e-10
depth: 0
894/1000	0.00	True	46.69 seconds
Estimated remaining time: 1 hours 29 minutes 32 seconds


0
1.2186203100226885e-09
depth: 0
895/1000	0.00	True	50.73 seconds
Estimated remaining time: 1 hours 28 minutes 41 seconds


0
4.648337

6.127639543625036e-11
depth: 0
953/1000	0.00	True	34.43 seconds
Estimated remaining time: 0 hours 39 minutes 37 seconds


0
2.0673403332570795e-09
depth: 0
954/1000	0.00	True	36.14 seconds
Estimated remaining time: 0 hours 38 minutes 46 seconds


0
7.533750738483473e-11
depth: 0
955/1000	0.00	True	31.63 seconds
Estimated remaining time: 0 hours 37 minutes 54 seconds


0
4.1057557740262784e-11
depth: 0
956/1000	0.00	True	45.94 seconds
Estimated remaining time: 0 hours 37 minutes 3 seconds


0
0.00032860808704435743
1
4.227886215372171e-11
depth: 1
957/1000	0.00	True	74.55 seconds
Estimated remaining time: 0 hours 36 minutes 14 seconds


0
1.8179674012576662e-11
depth: 0
958/1000	0.00	True	39.88 seconds
Estimated remaining time: 0 hours 35 minutes 23 seconds


0
5.919196546926368e-07
1
5.919173483519802e-07
2
5.919134543050104e-07
3
5.919172954000605e-07
4
9.954655555095044e-11
depth: 4
959/1000	0.00	True	385.21 seconds
Estimated remaining time: 0 hours 34 minutes 47 seconds


0
1.726693

In [2]:
############ Generating data and solve with the new double measurement method and many jumps ################

from datetime import datetime

ens = 500
        
step = 3
total_time = 0
data = []
errs = np.zeros(ens)
tms = np.zeros(ens)
results = np.zeros(ens, dtype='bool')

for i in range(ens):
    start = time.time()
    
    td = (30 + np.random.rand()*10)*365 
    tm = np.random.rand()*15
    
    tms[i] = tm

    '''
    #Njumps = 5 + int(np.random.rand()*15)
    Njumps = 4
    Rd = 100 + 2*(np.random.rand(Njumps)-0.5)*70
    Tjumps = np.sort(np.random.rand(Njumps))*td*0.8
    Tjumps[0] = 0
    '''
    
    
    # Generating 3 jumps within 0.9td and 1 jumps between 0.9td and td
    Njumps_first = 3
    Njumps_last = 1
    
    Rd = 100 + 2*(np.random.rand(Njumps_first+Njumps_last)-0.5)*70

    Tjumps_first = np.sort(np.random.rand(Njumps_first)) * td * 0.9  # Generates 3 jumps within 0.9td
   
    Tjumps_last = np.random.rand(Njumps_last) * (0.05 * td) + 0.9 * td  # Generates 1 jump between 0.9td and 0.95td

    Tjumps = np.concatenate((Tjumps_first, Tjumps_last))  # Combines the jumps into a single array

    # Sorts the jumps in ascending order
    Tjumps = np.sort(Tjumps)
    Tjumps[0] = 0
    
    
    
    
    t, R1t, R2t = Simulator_v3(td, td+tm+step,
                               100, True, Rd, Tjumps)

    Rt11 = np.interp(td+tm, t, R1t)
    Rt21 = np.interp(td+tm, t, R2t)
    Rt12 = R1t[-1]
    Rt22 = R2t[-1]

    sol = inverseProblemSolver3_DoubleMeasurement(Rt11, Rt21, Rt12, Rt22, step)
    
    errs[i] = np.abs(tm-sol.x[1])
    tms[i] = tm
    results[i] = sol.success
    
    s = {'td':td, 'tm':tm, 'Rd':Rd, 'Tjumps':Tjumps, 'sol':sol}
    
    data.append(s)
    
    stop = time.time()
    print('{}/{}\t{:.2f}\t{}\t{:.2f} seconds'.format(i + 1, ens, errs[i], results[i], stop-start))
    total_time += stop-start
        
    remaining = total_time * (ens - (i+1))/(i+1)
    remtime = [int(remaining//3600), int((remaining%3600)//60), int(remaining%60)]
    print('Estimated remaining time: {} hours {} minutes {} seconds\n\n'.format(remtime[0], remtime[1], remtime[2]))

    
now = datetime.now()
np.save('Data_report_no_error_v2_double_measurement_{}_{}_{}_{}_{}_{}.npy'.format(now.year, now.month, now.day, now.hour, now.minute, now.second), data)

%matplotlib qt
plt.plot(tms, np.abs(errs), '.')
plt.xlabel('Time difference between death and measurement [days]')
plt.ylabel('Estimation error [days]')
plt.grid()
plt.show()

plt.show()

0
1.424954813248645e-06
1
1.4249532245457802e-06
2
1.4249533188551877e-06
3
1.4249535373637126e-06
4
1.424955213364855e-06
5
1.4249543542415009e-06
6
1.4249579963172522e-06
depth: 6
1/500	0.04	True	341.72 seconds
Estimated remaining time: 47 hours 21 minutes 59 seconds


0
8.587834318291237e-07
1
8.581483274743841e-07
2
8.580864042121278e-07
3


KeyboardInterrupt: 